# イールドカーブ主成分分析 - データ復元と誤差分析

## 概要
このノートブックでは、国債金利データに対して主成分分析（PCA）を実行し、指定した主成分数でデータを復元して誤差を可視化します。

## 機能
1. 過去N日分のデータから主成分ベクトルを算出
2. 残存年限の和集合を取り、スプライン補間でデータを整形
3. 指定した複数日付でデータ復元を実行
4. 元データと復元データの誤差をグラフ表示

In [1]:
# 必要なライブラリのインポート
import os
import sys
import requests
from datetime import datetime, timedelta
from typing import List, Dict, Tuple
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from sklearn.decomposition import PCA
from dotenv import load_dotenv

# プロジェクトルートをパスに追加
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, project_root)

# 環境変数読み込み
load_dotenv(os.path.join(project_root, '.env'))

# Supabase設定
SUPABASE_URL = os.getenv('SUPABASE_URL')
SUPABASE_KEY = os.getenv('SUPABASE_KEY')

# 日本語フォント設定
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Hiragino Sans', 'Yu Gothic', 'Meirio']
plt.rcParams['axes.unicode_minus'] = False

print(f"✅ 環境設定完了")
print(f"Supabase URL: {SUPABASE_URL[:30]}..." if SUPABASE_URL else "❌ SUPABASE_URL not set")

✅ 環境設定完了
Supabase URL: https://yfravzuebsvkzjnabalj.s...


## パラメータ設定

以下のパラメータを調整してください：
- `lookback_days`: 主成分分析に使用する過去の営業日数
- `analysis_date`: 主成分分析を実行する基準日
- `reconstruction_dates`: 復元誤差を計算する日付のリスト
- `n_components`: 復元に使用する主成分数

In [2]:
# ===== ユーザー設定パラメータ =====

# 主成分分析設定
lookback_days = 60  # 過去何営業日分のデータを使うか
analysis_date = '2024-09-01'  # 分析基準日（YYYY-MM-DD形式）
n_components = 3  # 使用する主成分数

# 復元検証日（複数指定可能）
reconstruction_dates = [
    '2024-09-01',
    '2024-09-05',
    '2024-09-10'
]

## データ取得関数（同期版）

In [3]:
def get_yield_data_for_date(date: str) -> pd.DataFrame:
    """
    指定日のイールドカーブデータを取得
    
    Parameters:
    -----------
    date : str
        取得日付（YYYY-MM-DD形式）
    
    Returns:
    --------
    pd.DataFrame
        カラム: maturity (残存年限), yield_rate (利回り)
    """
    headers = {
        'apikey': SUPABASE_KEY,
        'Authorization': f'Bearer {SUPABASE_KEY}',
        'Content-Type': 'application/json'
    }
    
    params = {
        'select': 'trade_date,due_date,ave_compound_yield',
        'trade_date': f'eq.{date}',
        'ave_compound_yield': 'not.is.null',
        'due_date': 'not.is.null',
        'order': 'due_date.asc',
        'limit': '1000'
    }
    
    try:
        response = requests.get(
            f"{SUPABASE_URL}/rest/v1/bond_data",
            params=params,
            headers=headers
        )
        
        if response.status_code != 200 or not response.json():
            return pd.DataFrame(columns=['maturity', 'yield_rate'])
        
        # 残存年限を計算
        data_list = []
        for row in response.json():
            try:
                trade_dt = datetime.strptime(row['trade_date'], '%Y-%m-%d')
                due_dt = datetime.strptime(row['due_date'], '%Y-%m-%d')
                years_to_maturity = (due_dt - trade_dt).days / 365.25
                
                if years_to_maturity > 0 and row['ave_compound_yield'] is not None:
                    data_list.append({
                        'maturity': round(years_to_maturity, 3),
                        'yield_rate': float(row['ave_compound_yield'])
                    })
            except (ValueError, TypeError):
                continue
        
        return pd.DataFrame(data_list)
    
    except Exception as e:
        print(f"エラー ({date}): {e}")
        return pd.DataFrame(columns=['maturity', 'yield_rate'])


def get_available_dates(start_date: str, end_date: str, limit: int = 1000) -> List[str]:
    """
    指定期間内の利用可能な日付を取得
    
    Parameters:
    -----------
    start_date : str
        開始日（YYYY-MM-DD形式）
    end_date : str
        終了日（YYYY-MM-DD形式）
    limit : int
        取得する最大件数
    
    Returns:
    --------
    List[str]
        利用可能な日付のリスト（降順）
    """
    headers = {
        'apikey': SUPABASE_KEY,
        'Authorization': f'Bearer {SUPABASE_KEY}',
        'Content-Type': 'application/json'
    }
    
    params = {
        'select': 'trade_date',
        'trade_date': f'gte.{start_date}',
        'order': 'trade_date.desc',
        'limit': str(limit)
    }
    
    try:
        response = requests.get(
            f"{SUPABASE_URL}/rest/v1/bond_data",
            params=params,
            headers=headers
        )
        
        if response.status_code != 200:
            return []
        
        # 重複削除してソート
        dates = sorted(list(set([row['trade_date'] for row in response.json()])), reverse=True)
        
        # end_date以前でフィルタ
        dates = [d for d in dates if d <= end_date]
        
        return dates
    
    except Exception as e:
        print(f"日付取得エラー: {e}")
        return []

## データ前処理関数

In [4]:
def create_common_maturity_grid(daily_data: Dict[str, pd.DataFrame]) -> np.ndarray:
    """
    全日付の残存年限の和集合を取得してソート
    
    Parameters:
    -----------
    daily_data : Dict[str, pd.DataFrame]
        日付ごとのデータフレーム辞書
    
    Returns:
    --------
    np.ndarray
        共通年限軸（昇順）
    """
    all_maturities = set()
    for df in daily_data.values():
        if not df.empty:
            all_maturities.update(df['maturity'].values)
    
    return np.sort(np.array(list(all_maturities)))


def interpolate_yields(df: pd.DataFrame, common_grid: np.ndarray) -> np.ndarray:
    """
    スプライン補間を使って共通年限軸上の利回りを推定
    
    Parameters:
    -----------
    df : pd.DataFrame
        ある日付のデータ（maturity, yield_rate）
    common_grid : np.ndarray
        共通年限軸
    
    Returns:
    --------
    np.ndarray
        補間後の利回り配列
    """
    if df.empty or len(df) < 2:
        # データが不足している場合はNaNで埋める
        return np.full(len(common_grid), np.nan)
    
    # 重複削除・ソート
    df_sorted = df.sort_values('maturity').drop_duplicates('maturity')
    
    if len(df_sorted) < 2:
        return np.full(len(common_grid), np.nan)
    
    # CubicSplineで補間
    cs = CubicSpline(df_sorted['maturity'].values, 
                     df_sorted['yield_rate'].values,
                     extrapolate=False)  # 外挿は行わない
    
    return cs(common_grid)


def build_pca_matrix(daily_data: Dict[str, pd.DataFrame], 
                     dates: List[str],
                     common_grid: np.ndarray) -> Tuple[np.ndarray, List[str]]:
    """
    PCA用の行列を構築（各行が1日分、各列が年限）
    
    Parameters:
    -----------
    daily_data : Dict[str, pd.DataFrame]
        日付ごとのデータ
    dates : List[str]
        使用する日付リスト
    common_grid : np.ndarray
        共通年限軸
    
    Returns:
    --------
    Tuple[np.ndarray, List[str]]
        (PCA行列, 有効な日付リスト)
    """
    matrix_rows = []
    valid_dates = []
    
    for date in dates:
        if date not in daily_data:
            continue
        
        interpolated = interpolate_yields(daily_data[date], common_grid)
        
        # NaNが多すぎる行はスキップ
        if np.isnan(interpolated).sum() / len(interpolated) < 0.5:
            matrix_rows.append(interpolated)
            valid_dates.append(date)
    
    return np.array(matrix_rows), valid_dates

## 主成分分析と復元関数

In [5]:
def perform_pca(X: np.ndarray, n_components: int) -> Tuple[PCA, np.ndarray]:
    """
    主成分分析を実行
    
    Parameters:
    -----------
    X : np.ndarray
        入力データ行列（行=日付, 列=年限）
    n_components : int
        使用する主成分数
    
    Returns:
    --------
    Tuple[PCA, np.ndarray]
        (PCAオブジェクト, NaN補完後のデータ)
    """
    # NaNを列平均で補完
    X_filled = X.copy()
    col_means = np.nanmean(X, axis=0)
    for i in range(X.shape[1]):
        X_filled[np.isnan(X_filled[:, i]), i] = col_means[i]
    
    # PCA実行
    pca = PCA(n_components=n_components)
    pca.fit(X_filled)
    
    return pca, X_filled


def reconstruct_data(pca: PCA, 
                    original_data: np.ndarray,
                    n_components: int) -> np.ndarray:
    """
    主成分ベクトルを使ってデータを復元
    
    Parameters:
    -----------
    pca : PCA
        学習済みPCAオブジェクト
    original_data : np.ndarray
        元データ（1日分: 1次元配列）
    n_components : int
        使用する主成分数
    
    Returns:
    --------
    np.ndarray
        復元データ
    """
    # NaNを平均で補完
    data_filled = original_data.copy()
    if np.isnan(data_filled).any():
        data_filled[np.isnan(data_filled)] = np.nanmean(data_filled)
    
    # データを変換
    transformed = pca.transform(data_filled.reshape(1, -1))
    
    # n_components個の主成分のみ使用して逆変換
    transformed_partial = np.zeros_like(transformed)
    transformed_partial[0, :n_components] = transformed[0, :n_components]
    
    reconstructed = pca.inverse_transform(transformed_partial)
    
    return reconstructed[0]

## メイン処理実行

In [6]:
# 分析基準日から過去のlookback_days営業日を取得
start_date = (datetime.strptime(analysis_date, '%Y-%m-%d') - timedelta(days=lookback_days * 2)).strftime('%Y-%m-%d')
available_dates = get_available_dates(start_date, analysis_date)

# 必要な営業日数を取得
training_dates = available_dates[:lookback_days]
print(f"主成分分析に使用する日付: {len(training_dates)}日分")
print(f"期間: {training_dates[-1]} ~ {training_dates[0]}")

主成分分析に使用する日付: 0日分


IndexError: list index out of range

In [ ]:
# 学習データ取得
daily_data = {}
for i, date in enumerate(training_dates, 1):
    df = get_yield_data_for_date(date)
    if not df.empty:
        daily_data[date] = df
    
    if i % 10 == 0:
        print(f"  データ取得中... {i}/{len(training_dates)}")

print(f"✅ データ取得完了: {len(daily_data)}日分")

In [ ]:
# 共通年限軸の作成
common_grid = create_common_maturity_grid(daily_data)
print(f"共通年限軸: {len(common_grid)}個の年限")
print(f"範囲: {common_grid.min():.2f}年 ~ {common_grid.max():.2f}年")

In [ ]:
# PCA行列の構築
X, valid_dates = build_pca_matrix(daily_data, training_dates, common_grid)
print(f"PCA行列サイズ: {X.shape}（{X.shape[0]}日 x {X.shape[1]}年限）")

In [ ]:
# 主成分分析実行
pca, X_filled = perform_pca(X, n_components=min(n_components, X.shape[1]))

print(f"\n主成分分析結果:")
print(f"使用主成分数: {pca.n_components_}")
print(f"累積寄与率: {pca.explained_variance_ratio_.cumsum()[-1]:.2%}")
print(f"\n各主成分の寄与率:")
for i, ratio in enumerate(pca.explained_variance_ratio_, 1):
    print(f"  PC{i}: {ratio:.2%}")

## 復元と誤差計算

In [ ]:
# 復元検証日のデータを取得
reconstruction_results = {}

for date in reconstruction_dates:
    # 元データ取得
    df_original = get_yield_data_for_date(date)
    
    if df_original.empty:
        print(f"警告: {date} のデータが見つかりません")
        continue
    
    # 共通年限軸に補間
    original_interpolated = interpolate_yields(df_original, common_grid)
    
    # データ復元
    reconstructed = reconstruct_data(pca, original_interpolated, n_components)
    
    # 誤差計算
    error = original_interpolated - reconstructed
    
    reconstruction_results[date] = {
        'original': original_interpolated,
        'reconstructed': reconstructed,
        'error': error
    }
    
    print(f"{date}: 平均絶対誤差 = {np.nanmean(np.abs(error)):.4f}%")

## 誤差グラフの可視化

In [ ]:
# グラフ描画
plt.figure(figsize=(14, 6))

colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6', '#1abc9c']

for i, (date, result) in enumerate(reconstruction_results.items()):
    color = colors[i % len(colors)]
    plt.plot(common_grid, result['error'], 
             label=f'{date}', 
             color=color, 
             linewidth=2,
             marker='o',
             markersize=4,
             alpha=0.8)

plt.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
plt.xlabel('残存年限（年）', fontsize=12)
plt.ylabel('誤差（元データ - 復元データ）[%]', fontsize=12)
plt.title(f'主成分分析による復元誤差（使用主成分数: {n_components}）', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n=== 分析完了 ===")
print(f"学習期間: {training_dates[-1]} ~ {training_dates[0]} ({len(training_dates)}日)")
print(f"使用主成分数: {n_components}")
print(f"累積寄与率: {pca.explained_variance_ratio_.cumsum()[-1]:.2%}")